In [53]:
import os 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [26]:
# Macromodelos son aquellos con TS diferente (y todo lo demás también)
macromodels = ["4_01","4_02"]
# Models comparten TS, pero no hiperparámetros. 
models = ["m1","m2"]

In [28]:
# Esta lista acumula la agregación para cada macromodels
dataframes_finales = []

In [29]:
for macromodel in macromodels:
    # Esta lista va a tener los resultados para cada modelo de un macromodelo
    dataframes_subfinales = []
    for model in models:
        pred_dir = f"{macromodel}/preds/"
        pred_files = [f"{macromodel}/preds/{model}/" + i for i in os.listdir(pred_dir + model)]
        # En este for se combinan todas las predicciones
        for n, pred_file in enumerate(pred_files):
            if n == 0:
                d = pd.read_csv(pred_file, sep = "\t").drop(columns = ["foto_mes"]).rename(columns = {"prob":pred_file})
            else:
                d2 = pd.read_csv(pred_file, sep = "\t").drop(columns = ["foto_mes"]).rename(columns = {"prob":pred_file})
                d = pd.merge(d,d2, on = "numero_de_cliente", how = "outer")

        d = d.transpose()
        d.columns = d.loc["numero_de_cliente"].astype(int).astype(str)
        d = d.drop("numero_de_cliente")
        d =  d.rank(ascending = False,axis = 1)
        
        # Estoy promediando las semillas
        d_final = pd.DataFrame(d.mean(axis = 0)).rename(columns = {
            0 : f"{macromodel}/{model}"})
        
        dataframes_subfinales.append(d_final)
    
    marcomodel_final = dataframes_subfinales[0].merge(dataframes_subfinales[1],
                    left_index = True, right_index = True)
        
        # Estoy agarrando el máximo dentro de los modelos 
    marcomodel_final = pd.DataFrame(marcomodel_final.max(axis = 1)).rename(columns = {
            0 : f"{macromodel}"})
    dataframes_finales.append(marcomodel_final)

In [34]:
df = dataframes_finales[0]
for df2 in dataframes_finales[1:]:
    df = df.merge(df2, left_index = True, right_index = True)

In [69]:
df_entrega = pd.DataFrame(df.mean(axis = 1)).sort_values(0).copy()

In [75]:
n_envios =np.linspace(4000,16000,33)
for n in n_envios:
    n = int(n)
    nombre_entrega = f"401y2_model1&2_rank_{n}e_mean_max_mean.csv"
    df_entrega_envio = df_entrega.copy()
    df_entrega_envio["Predicted"] = 0
    df_entrega_envio.loc[0:n-1, "Predicted"] = 1
    df_entrega_envio = df_entrega_envio.drop(columns = [0])
    df_entrega_envio.to_csv("entregas/"+nombre_entrega)

/tmp/ipykernel_10126/319758513.py:7: FutureWarning: Slicing a positional slice with .loc is not supported, and will raise TypeError in a future version.  Use .loc with labels or .iloc with positions instead.
  df_entrega_envio.loc[0:n-1, "Predicted"] = 1


array([ 4000.,  4375.,  4750.,  5125.,  5500.,  5875.,  6250.,  6625.,
        7000.,  7375.,  7750.,  8125.,  8500.,  8875.,  9250.,  9625.,
       10000., 10375., 10750., 11125., 11500., 11875., 12250., 12625.,
       13000., 13375., 13750., 14125., 14500., 14875., 15250., 15625.,
       16000.])